<a href="https://colab.research.google.com/github/aharonsh/data-engineering-practice/blob/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Create Vertex AI Vector Search index


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fvector_search%2Fsdk_vector_search_for_indexing.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This example demonstrates how to use the Vertex AI ANN Service. It's a high scale, low latency solution, to find similar vectors (or more specifically "embeddings") for a large corpus. Moreover, it's a fully managed offering, further reducing operational overhead. The Vertex AI ANN Service is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

Learn more about [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview).

### Objective

In this notebook, you learn how to create Approximate Nearest Neighbor (ANN) Index, query against indexes, and validate the performance of the index.

This tutorial uses the following Vertex AI services:

- Vertex AI Vector Search

The steps performed include:

* Create ANN Index and Brute Force Index.
* Create an IndexEndpoint with VPC Network.
* Deploy ANN Index and Brute Force Index.
* Perform online query.
* Compute recall.


### Dataset

The dataset used for this tutorial is the [GloVe dataset](https://nlp.stanford.edu/projects/glove/).

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus. The resulting representations showcase interesting linear substructures of the word vector space.


## Get started

### Install Vertex AI SDK for Python and other required packages


In [8]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage \
                         grpcio-tools \
                         h5py

  Using cached google_cloud_storage-3.4.0-py3-none-any.whl.metadata (13 kB)


### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [9]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information
Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "rons-labs-33840"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Prepare a VPC network
To reduce any network overhead that might lead to unnecessary increase in overhead latency, it's best to call the ANN endpoints from your VPC via a direct [VPC Peering](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) connection.
  * The following section describes how to setup a VPC Peering connection if you don't have one.
  * This is a one-time initial setup task. You can also reuse existing VPC network and skip this section.

In [3]:
VPC_NETWORK = "colab-research-vpc"  # @param {type:"string"}

PEERING_RANGE_NAME = "ann-haystack-range"

In [5]:
import os

# Remove the if condition to run the encapsulated code
if not os.getenv("IS_TESTING"):
    # Create a VPC network
    ! gcloud compute networks create {VPC_NETWORK} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

    # Add necessary firewall rules
    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-icmp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow icmp

    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-internal --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-rdp --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

    ! gcloud compute firewall-rules create {VPC_NETWORK}-allow-ssh --network {VPC_NETWORK} --priority 65534 --project {PROJECT_ID} --allow tcp:22

    # Reserve IP range
    ! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={VPC_NETWORK} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range"

    # Set up peering with service networking
    # Your account must have the "Compute Network Admin" role to run the following.
    ! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={VPC_NETWORK} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

ERROR: (gcloud.compute.networks.create) Could not fetch resource:
 - The resource 'projects/rons-labs-33840/global/networks/colab-research-vpc' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/rons-labs-33840/global/firewalls/colab-research-vpc-allow-icmp' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/rons-labs-33840/global/firewalls/colab-research-vpc-allow-internal' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/rons-labs-33840/global/firewalls/colab-research-vpc-allow-rdp' already exists

ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/rons-labs-33840/global/firewalls/colab-research-vpc-allow-ssh' already exists

Created [https://www.googleapis.com/compute/v1/projects/rons-labs-33840/global/addresses/ann-haystack-range].
Operation "opera

* Authentication: Rerun the `gcloud auth login` command in the Vertex AI Workbench notebook terminal when you are logged out and need the credential again.

**WARNING:** The `MatchingIndexEndpoint.match` method (to create online queries against your deployed index) has to be executed in a Vertex AI Workbench notebook instance that is created with the following requirements:
  * **In the same region where your ANN service is deployed** (for example, if you set `LOCATION = "us-central1"` as same as the tutorial, the notebook instance has to be in `us-central1`).
  
  * **Make sure you select the VPC network you created for ANN service** (instead of using the "default" one). That is, you have to create a new notebook instance that uses the VPC network you created earlier. Run the rest of the tutorial from that notebook instance.
  * If you run it in a Colab or a Vertex AI Workbench notebook in a different VPC network or region, "Create Online Queries" section is expected to fail.

## Import the required libraries


In [6]:
import json

import h5py
from google.cloud import aiplatform
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import \
    Namespace

## Create a cloud storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [7]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [8]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://your-bucket-name-rons-labs-33840-unique/...


## Initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

In [9]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Prepare the data

The GloVe dataset consists of a set of pre-trained embeddings. The embeddings are split into a "train" split, and a "test" split. You create a vector search index from the "train" split, and use the embedding vectors in the "test" split as query vectors to test the vector search index.

**Note:** While the data split uses the term "train", these are pre-trained embeddings and therefore are ready to be indexed for search. The terms "train" and "test" split are used just to be consistent with machine learning terminology.

Download the GloVe dataset.


In [10]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5 .

Copying gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][462.9 MiB/462.9 MiB]   30.3 MiB/s                                   
Operation completed over 1 objects/462.9 MiB.                                    


Read the data into memory.


In [11]:
# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 10

h5 = h5py.File("glove-100-angular.hdf5", "r")
train = h5["train"]
test = h5["test"]

In [12]:
# check the first record
train[0]

array([-0.11333  ,  0.48402  ,  0.090771 , -0.22439  ,  0.034206 ,
       -0.55831  ,  0.041849 , -0.53573  ,  0.18809  , -0.58722  ,
        0.015313 , -0.014555 ,  0.80842  , -0.038519 ,  0.75348  ,
        0.70502  , -0.17863  ,  0.3222   ,  0.67575  ,  0.67198  ,
        0.26044  ,  0.4187   , -0.34122  ,  0.2286   , -0.53529  ,
        1.2582   , -0.091543 ,  0.19716  , -0.037454 , -0.3336   ,
        0.31399  ,  0.36488  ,  0.71263  ,  0.1307   , -0.24654  ,
       -0.52445  , -0.036091 ,  0.55068  ,  0.10017  ,  0.48095  ,
        0.71104  , -0.053462 ,  0.22325  ,  0.30917  , -0.39926  ,
        0.036634 , -0.35431  , -0.42795  ,  0.46444  ,  0.25586  ,
        0.68257  , -0.20821  ,  0.38433  ,  0.055773 , -0.2539   ,
       -0.20804  ,  0.52522  , -0.11399  , -0.3253   , -0.44104  ,
        0.17528  ,  0.62255  ,  0.50237  , -0.7607   , -0.071786 ,
        0.0080131, -0.13286  ,  0.50097  ,  0.18824  , -0.54722  ,
       -0.42664  ,  0.4292   ,  0.14877  , -0.0072514, -0.1648

### Save the train split in JSONL format.

The data must be formatted in JSONL format, which means each embedding dictionary is written as a JSON string on its own line.

Additionally, to demonstrate the filtering functionality, the `restricts` key is set such that each embedding has a different `class`, `even` or `odd`. These are used during the later matching step to filter for results.
For additional information of filtering, see [Filter vector matches](https://cloud.google.com/vertex-ai/docs/vector-search/filtering).

In [13]:
with open("glove100.json", "w") as f:
    embeddings_formatted = [
        json.dumps(
            {
                "id": str(index),
                "embedding": [str(value) for value in embedding],
                "restricts": [
                    {
                        "namespace": "class",
                        "allow": ["even" if index % 2 == 0 else "odd"],
                    }
                ],
            }
        )
        + "\n"
        for index, embedding in enumerate(train)
    ]
    f.writelines(embeddings_formatted)

Upload the training data to GCS.

In [14]:
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/vector_search/initial/"
! gsutil cp glove100.json {EMBEDDINGS_INITIAL_URI}

Copying file://glove100.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][  1.4 GiB/  1.4 GiB]   23.0 MiB/s                                   
Operation completed over 1 objects/1.4 GiB.                                      


## Create Indexes


In [15]:
# set no.of dimensions for your embeddings
DIMENSIONS = 100
# set the dispaly name for ann index
DISPLAY_NAME = "glove_100_1"
# set the display name for brute force index
DISPLAY_NAME_BRUTE_FORCE = DISPLAY_NAME + "_brute_force"

### Create ANN Index (for Production Usage)

Create the ANN index configuration:

To learn more about configuring the index, see [Input data format and structure](https://cloud.google.com/vertex-ai/docs/vector-search/setup/format-structure).


In [16]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    description="Glove 100 ANN index",
    labels={"label_name": "label_value"},
)

In [17]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/249093867948/locations/us-central1/indexes/7028555512790646784'

Using the resource name, you can retrieve an existing Vector Search Index.

In [18]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

### Create Brute Force Index (for Ground Truth)

The brute force index uses a naive brute force method to find the nearest neighbors. This method isn't fast or efficient. Hence, brute force indices are not recommended for production usage. They are to be used to find the "ground truth" set of neighbors so that the "ground truth" set can be used to measure recall of the indices being tuned for production usage. To ensure an "apples to apples" comparison, the `distanceMeasureType` and `dimensions` of the brute force index should match those of the production indices being tuned.

Create the brute force index configuration:

In [19]:
brute_force_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
    display_name=DISPLAY_NAME_BRUTE_FORCE,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Glove 100 index (brute force)",
    labels={"label_name": "label_value"},
)

In [20]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = brute_force_index.resource_name
INDEX_BRUTE_FORCE_RESOURCE_NAME

'projects/249093867948/locations/us-central1/indexes/7031933212511174656'

In [21]:
brute_force_index = aiplatform.MatchingEngineIndex(
    index_name=INDEX_BRUTE_FORCE_RESOURCE_NAME
)

## Update Indexes

Create an incremental data file.


In [22]:
with open("glove100_incremental.json", "w") as f:
    index = 0
    f.write(
        json.dumps(
            {
                "id": str(index),
                "embedding": [str(0) for _ in train[index]],
                "restricts": [
                    {
                        "namespace": "class",
                        "allow": ["even" if index % 2 == 0 else "odd"],
                    }
                ],
            }
        )
        + "\n"
    )

Copy the incremental data file to a new subdirectory.


In [23]:
EMBEDDINGS_UPDATE_URI = f"{BUCKET_URI}/vector-search/incremental/"

In [24]:
! gsutil cp glove100_incremental.json {EMBEDDINGS_UPDATE_URI}

Copying file://glove100_incremental.json [Content-Type=application/json]...
\
Operation completed over 1 objects/585.0 B.                                      


Create update index request.


In [25]:
tree_ah_index = tree_ah_index.update_embeddings(
    contents_delta_uri=EMBEDDINGS_UPDATE_URI,
)

In [26]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/249093867948/locations/us-central1/indexes/7028555512790646784'

## Create an IndexEndpoint in your VPC Network

In [27]:
# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
# Get the full network resource name
VPC_NETWORK_FULL = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, VPC_NETWORK)
VPC_NETWORK_FULL

'projects/249093867948/global/networks/colab-research-vpc'

In [28]:
# Create your IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index_endpoint_for_demo",
    description="index endpoint description",
    network=VPC_NETWORK_FULL,
)

In [29]:
INDEX_ENDPOINT_NAME = my_index_endpoint.resource_name
INDEX_ENDPOINT_NAME

'projects/249093867948/locations/us-central1/indexEndpoints/6750423051428429824'

## Deploy Indexes

### Deploy ANN Index

In [30]:
# Set an id for your ann index deployment
DEPLOYED_INDEX_ID = "tree_ah_glove_deployed_unique"

In [31]:
# Deploy your ann index
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

[id: "tree_ah_glove_deployed_unique"
index: "projects/249093867948/locations/us-central1/indexes/7028555512790646784"
create_time {
  seconds: 1758445602
  nanos: 975917000
}
private_endpoints {
  match_grpc_address: "10.117.0.14"
}
index_sync_time {
  seconds: 1758445602
  nanos: 975917000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

### Deploy Brute Force Index

In [32]:
# Set an id for your brute force index deployment
DEPLOYED_BRUTE_FORCE_INDEX_ID = "glove_brute_force_deployed_unique"

In [33]:
# Deploy your brute force index
my_index_endpoint = my_index_endpoint.deploy_index(
    index=brute_force_index, deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID
)

my_index_endpoint.deployed_indexes

[id: "tree_ah_glove_deployed_unique"
index: "projects/249093867948/locations/us-central1/indexes/7028555512790646784"
create_time {
  seconds: 1758445602
  nanos: 975917000
}
private_endpoints {
  match_grpc_address: "10.117.0.14"
}
index_sync_time {
  seconds: 1758447328
  nanos: 242064000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
, id: "glove_brute_force_deployed_unique"
index: "projects/249093867948/locations/us-central1/indexes/7031933212511174656"
create_time {
  seconds: 1758447249
  nanos: 99946000
}
private_endpoints {
  match_grpc_address: "10.117.0.14"
}
index_sync_time {
  seconds: 1758447493
  nanos: 504964000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

## Create online queries

After you've built your indexes, you may query against the deployed index through the online querying gRPC API (Match service) within the virtual machine instances from the same region (for example, 'us-central1' in this tutorial).

The `filter` parameter is an optional way to filter for a subset of embeddings. In this case, only embeddings that have the `class` set as `even` are returned.

In [34]:
# Use match service with a test query
response = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test[:1].tolist(),
    num_neighbors=NUM_NEIGHBOURS,
    filter=[Namespace("class", ["even"])],
)

response

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:10.117.0.14:10000: Failed to connect to remote host: Timeout occurred: FD Shutdown"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:10.117.0.14:10000: Failed to connect to remote host: Timeout occurred: FD Shutdown"}"
>

### Compute recall

Use the deployed brute force Index as the ground truth to calculate the recall of ANN Index. You can run multiple queries in a single match call.

In [ ]:
# Retrieve nearest neighbors for both the tree-AH index and the brute force index
tree_ah_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test[:].tolist(),
    num_neighbors=NUM_NEIGHBOURS,
)
brute_force_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID,
    queries=test[:].tolist(),
    num_neighbors=NUM_NEIGHBOURS,
)

In [ ]:
# Calculate recall by determining how many neighbors were correctly retrieved as compared to the brute force option.
recalled_neighbors = 0
for tree_ah_neighbors, brute_force_neighbors in zip(
    tree_ah_response_test, brute_force_response_test
):
    tree_ah_neighbor_ids = [neighbor.id for neighbor in tree_ah_neighbors]
    brute_force_neighbor_ids = [neighbor.id for neighbor in brute_force_neighbors]

    recalled_neighbors += len(
        set(tree_ah_neighbor_ids).intersection(brute_force_neighbor_ids)
    )

recall = recalled_neighbors / len(
    [neighbor for neighbors in brute_force_response_test for neighbor in neighbors]
)

print("Recall: {}".format(recall))

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [35]:
delete_bucket = False

# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

# Delete indexes
tree_ah_index.delete()
brute_force_index.delete()

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}